In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

## 포지션-와이즈 피드 포워드 신경망(Position-wise FFNN)  

지금은 인코더를 설명하고 있지만, 포지션 와이즈 FFNN은 인코더와 디코더에서 공통적으로 가지고 있는 서브층입니다.  
포지션-와이즈 FFNN는 쉽게 말하면 완전 연결 FFNN(Fully-connected FFNN)이라고 해석할 수 있음. 앞서 인공 신경망은  
결국 벡터와 행렬 연산으로 표현될 수 있음을 배웠음. 아래는 포지션 와이즈 FFNN의 수식을 보여줌.  

<div align="left">
$FFNN(x) = \max(0, xW_1 + b_1)W_2 + b_2$
</div>

식을 그림으로 표현하면 아래와 같음.  

![fig_1](./images/positionwiseffnn.png)

여기서 $x$는 앞서 멀티 헤드 어텐션의 결과로 나온 $(seq\_len, d_{model})$의 크기를 가지는 행렬을 말함.  
가중치 행렬 $W_1$은 $(d_{model}, d_{ff})$의 크기를 가지고, 가중치 행렬 $W_{2}$은 $(d_{ff},d_{model})$의 크기를 가짐.  
논문에서 은닉층의 크기인 $d_ff$는 앞서 하이퍼파라미터를 정의할 때 언급했듯이 2,048의 크기를 가집니다.  

여기서 매개변수 $W_1,b_1,W_2,b_2$는 하나의 인코더 층 내에서는 다른 문장, 다른 단어들마다 정확하게 동일하게 사용됨.  
하지만 인코더 층마다는 다른 값을 가짐.  

![fig2](./images/transformer20.png)

위의 그림에서 좌측은 인코더의 입력을 벡터 단위로 봤을 때, 각 벡터들이 멀티 헤드 어텐션 층이라는   
인코더 내 첫번째 서브 층을 지난 FFNN을 통과하는 것을 보여줌. 이는 두번째 서브층인 Position-wise FFNN을 의미함.  
물론, 실제로 그림의 우측과 같이 행렬로 연산되는데, 두번째 서브층을 지난 인코더의 최종 출력은 여전히 인코더의  
입력의 크기였던 $(seq\_len, d_{model})$의 크기가 보존되고 있음. 하나의 인코더 층을 지난 이 행렬은 다음 인코더  
층으로 전달되고, 다음 층에서도 동일한 인코더 연산이 반복됩니다.  


In [2]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [3]:
n_batch,d_model,h=1, 512, 8
seq_len = 10
d_ff = 2048
pwff = PositionwiseFeedForward(d_model,d_ff,0)
X = Variable(torch.rand(n_batch, seq_len, d_model))
y= pwff(X)